In [43]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Column, String, Integer, Float, Date, Table, ForeignKey
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, relationship

import pandas as pd

In [44]:
credentials = pd.read_csv('../credentials/credentials.csv')

database_adress = credentials['Host'].iloc[0]
database_user = credentials['User'].iloc[0]
database_password = credentials['Password'].iloc[0]
database_name = credentials['Database'].iloc[0]
database_port = credentials['Port'].iloc[0]

print(database_user)

darth_vader


In [45]:
Base = automap_base()

# Create the middle Table movie_genres
movie_genres = Table(
    "movies_genres",
    Base.metadata,
    Column("movieId", ForeignKey("movies.movieId")),
    Column("genreId", ForeignKey("genres.genreId")),
)

#Create the class Genre with it's 2 properties genre_id and name
class Genre(Base):
   __tablename__ = 'genres' 
   genre_id = Column('genreId', Integer, primary_key = True)
   name = Column('genrename', String)

# Create the class Movie with it's 3 properites movie_id, title and release_year.
# Additionally connect the genres table with the movies table
class Movie(Base):
    __tablename__ = 'movies'

    movie_id = Column('movieid', Integer, primary_key = True)
    title = Column('title', String)
    release_year = Column('release_year', Integer)

    genres = relationship(Genre, secondary=movie_genres,
                            backref='genres', viewonly=True)
class Genome_Tag(Base):
    __tablename__ = 'genometags' 
    tag_id = Column('tagId', Integer, primary_key = True)
    tag = Column('tag', String)


class Link(Base):
    __tablename__ = 'links' 
    movie_id = Column('movieId', Integer, primary_key = True)
    imdb_id = Column('imdbId', Integer)
    tmdb_id = Column('tmdbId', Integer)

class Rating(Base):
    __tablename__ = 'ratings' 
    rating_id = Column('rating_id', Integer, primary_key = True)
    user_id = Column('userId', Integer)
    movie = relationship(Movie, backref='movie_id', viewonly=True)
    rating = Column('rating', Float)
    rating_date = Column('rating_date', Date)

class Tag(Base):
    __tablename__ = 'tags' 
    tag_id = Column('tag_id', Integer, primary_key = True)
    user_id = Column('userId', Integer)
    movie = relationship(Movie, backref='movieId', viewonly=True)
    tag = Column('tag', String)
    tag_date = Column('tag_date', Date) 

class Genome_Score(Base):
    __tablename__ = 'genomescores' 
    tag_id = Column('tagId', Integer, primary_key = True)
    movie = relationship(Movie, backref='movieId', viewonly=True)
    relevance = Column('relevance', Float)

In [46]:
#Build the connection with the database
engine = create_engine(f'mysql+pymysql://{database_user}:{database_password}@{database_adress}:{database_port}/{database_name}')
Base.prepare(autoload_with=engine)

session = Session(engine)

# Call the first Entry in the movies table and print out it's properties and all it's genres
u1 = session.query(Movie).first()
print (f'{u1.movie_id}, {u1.title}, {u1.release_year}')

for genre in list(u1.genres):
    print(genre.name)

# Call the first Entry in the genre table and print out it's properites
u2 = session.query(Genre).first()
print(f'{u2.genre_id}, {u2.name}')

u3 = session.query(Genome_Score).first()
print(u3.movie.title)

u4 = session.query(Tag).first()
print(u4.movie.title)

1, Toy Story (1995), 1995
Adventure
Animation
Children
Comedy
Fantasy
1, Action
Toy Story (1995)
Braveheart (1995)
